<a href="https://colab.research.google.com/github/BilalAhmed7072/BLIP-Model/blob/main/myCaptioningModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets bitsandbytes peft sentensepiece -q
!pip install timm

ERROR: Could not find a version that satisfies the requirement sentensepiece (from versions: none)
ERROR: No matching distribution found for sentensepiece


In [ ]:
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from datasets import load_dataset

In [ ]:
model_id =  "Salesforce/blip-image-captioning-base"
processor = BlipProcessor.from_pretrained(model_id)
model = BlipForConditionalGeneration.from_pretrained(model_id)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("jxie/flickr8k", split = "train")
dataset = dataset.train_test_split(test_size= 0.1)

README.md:   0%|          | 0.00/687 [00:00<?, ?B/s]

data/train-00000-of-00002-2f8f6bfa852eac(…):   0%|          | 0.00/414M [00:00<?, ?B/s]

data/train-00001-of-00002-2173151d8cd6c7(…):   0%|          | 0.00/414M [00:00<?, ?B/s]

data/validation-00000-of-00001-7025a2b59(…):   0%|          | 0.00/138M [00:00<?, ?B/s]

data/test-00000-of-00001-42a2661d12c73e4(…):   0%|          | 0.00/137M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset["train"]
train_dataset[2]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x333>,
 'caption_0': 'A dirty jeep is stuck in the mud .',
 'caption_1': 'A Jeep goes off-roading in the woods with muddy trails making it dirty .',
 'caption_2': 'A muddy jeep driving through the forest .',
 'caption_3': 'A muddy truck in the woods .',
 'caption_4': 'The vehicle is driving through mud surrounded by pine trees .'}

In [ ]:
test_dataset = dataset["test"]
test_dataset[1]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=375x500>,
 'caption_0': 'A man in a red shirt shopping for candy',
 'caption_1': 'A person wearing a homemade storm trooper helmet looking at a display of candy .',
 'caption_2': 'A young man wearing a large space helmet is looking at candy in a market .',
 'caption_3': 'Man with storm trooper head gear on shops for candy .',
 'caption_4': 'Person standing in front of colorful display of candy .'}

In [ ]:
def preprocess (example):
  input = processor(images=example["image"],
                    text = example["caption_0"],
                    return_tensors= "pt",
                    padding = "max_length",
                    truncation = True)
  return{ "pixel_values":input["pixel_values"].squeeze(), "labels":input["input_ids"].squeeze()}



In [ ]:
processed_dataset = dataset.map(preprocess, remove_columns=dataset['train'].column_names)

Map:   0%|          | 0/5400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig , get_peft_model
config = LoraConfig(r=8,
    lora_alpha=16,
    target_modules=["projection"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM")

In [ ]:
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 147,456 || all params: 247,561,532 || trainable%: 0.0596


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./blip-lora-flickr8k",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,  # just 1 epoch for learning
    learning_rate=5e-5,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["test"],
)
trainer.train()


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

In [ ]:
from PIL import Image

image = dataset["test"][0]["image"]
inputs = processor(image, return_tensors="pt")

out = model.generate(**inputs, max_new_tokens=20)
print("Generated Caption:", processor.decode(out[0], skip_special_tokens=True))


In [ ]:
!pip install gradio -q
import gradio as gr

def caption_image(image):
    inputs = processor(image, return_tensors="pt")
    out = model.generate(**inputs, max_new_tokens=20)
    return processor.decode(out[0], skip_special_tokens=True)

demo = gr.Interface(fn=caption_image, inputs="image", outputs="text")
demo.launch()
